# Import environment

In [ ]:
from datasets import Dataset
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [39]:
# Convert JSON file to CSV file
df = pd.read_json('~/dataset/psy.json')
ds = Dataset.from_pandas(df)

In [40]:
ds[:3]

{'instruction': ["A major fire has broken out in the city center. A user who lives in the city with a tone of voice of concerned, empathetic, and anxious on social media is worried and sympathetic about this fire. They now send a new comment: 'I'm just hoping everything turns out okay, but it doesn't feel like it.' With your professional knowledge in psychology and social psychology, please analyze if the language use of the new comment conforms with the previous comments regarding tone of voice. Moreover, determine if their reaction aligns with their attitude towards the event. Their previous social media comments are provided in the input section.",
  "A major political event has occurred in Washington D.C. A user from D.C. with a tone of voice of passionate, engaged, and empathetic on social media is enthusiastic and supportive about this major political event. This user's previous real social media comments are 'This is a historic moment for our country and it's inspiring to see ho

# Processing data set

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(os.path.expanduser('~/LLM-Research/Meta-Llama-3-8B-Instruct'), use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [42]:
tokenizer.pad_token, tokenizer.pad_token_id, tokenizer.eos_token_id

('<|end_of_text|>', 128001, 128001)

In [43]:
def process_func(example):
    MAX_LENGTH = 384   
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|start_header_id|>user<|end_header_id|>\n\n{example['instruction'] + example['input']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n", add_special_tokens=False)  
    response = tokenizer(f"{example['output']}<|eot_id|>", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # Because we also need to pay attention to eos token, we add 1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # Make a truncation
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [44]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|███████████████████████| 25879/25879 [00:19<00:00, 1299.62 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 25879
})

In [45]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))

<|start_header_id|>user<|end_header_id|>

A major fire has broken out in the city center. A user who lives in the city with a tone of voice of concerned, empathetic, and anxious on social media is worried and sympathetic about this fire. They now send a new comment: 'I'm just hoping everything turns out okay, but it doesn't feel like it.' With your professional knowledge in psychology and social psychology, please analyze if the language use of the new comment conforms with the previous comments regarding tone of voice. Moreover, determine if their reaction aligns with their attitude towards the event. Their previous social media comments are provided in the input section.The sight of all these people trapped in smoke is heartbreaking. We need to keep our community safe, no matter what. Let's make sure everyone is accounted for, together.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The tone of voice of this tweet aligns with the previous tweets as concerned and empathetic. 

In [46]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'The tone of voice of this new comment aligns with the previous comments as passionate, engaged, and empathetic. The user expresses a desire to support and uphold the values of the event, which is consistent with their previous attitude of enthusiasm and support. This shows consistency with their previous attitude of being enthusiastic and supportive.<|eot_id|><|end_of_text|>'

# Define Model

In [47]:
import torch

model = AutoModelForCausalLM.from_pretrained(os.path.expanduser('~/LLM-Research/Meta-Llama-3-8B-Instruct'), device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards: 100%|██████████████████| 4/4 [00:05<00:00,  1.45s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [48]:
model.enable_input_require_grads() # This method is performed when the gradient checkpoint is enabled

In [49]:
model.dtype

torch.bfloat16

# Lora 

In [50]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # Training pattern
    r=8, 
    lora_alpha=32, # Lora alaph
    lora_dropout=0.1# Dropout Ratio
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'v_proj', 'up_proj', 'q_proj', 'k_proj', 'o_proj', 'gate_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [51]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/home/fangjianjie/test/autodl-tmp/LLM-Research/Meta-Llama-3-8B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'v_proj', 'up_proj', 'q_proj', 'k_proj', 'o_proj', 'gate_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [52]:
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.26047588741133265


# Configure training parameters

In [53]:
args = TrainingArguments(
    output_dir="./output/psy",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()

#  Save LoRA and tokenizer results


In [ ]:
peft_model_id="./psy"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)